# Backtracking - Constraint Satisfaction and Combinatorial Problems

## Learning Objectives
- Master the backtracking algorithmic paradigm
- Understand when and how to apply backtracking to complex problems
- Practice the choose-explore-unchoose pattern
- Learn optimization techniques and pruning strategies

## Key Patterns Covered
1. **Permutations**: Generate all arrangements of elements
2. **Combinations**: Choose k elements from n elements
3. **Subsets**: Generate all possible subsets (power set)
4. **Constraint Satisfaction**: N-Queens, Sudoku solving
5. **Path Finding**: Word search, maze solving
6. **Partitioning**: Palindrome partitioning, expression evaluation

---

## Backtracking Fundamentals

### The Backtracking Template:
```python
def backtrack(current_state):
    if is_solution(current_state):
        process_solution(current_state)
        return
    
    for choice in get_choices(current_state):
        if is_valid_choice(choice, current_state):
            # Choose
            make_choice(choice, current_state)
            
            # Explore
            backtrack(current_state)
            
            # Unchoose (backtrack)
            undo_choice(choice, current_state)
```

### When to Use Backtracking:
- Generate all possibilities (permutations, combinations, subsets)
- Constraint satisfaction problems (N-Queens, Sudoku)
- Path finding with constraints
- Optimization problems requiring exhaustive search

## Problem 1: Generate Permutations

**Problem**: Generate all possible permutations of a given array.

**Approach**: Use backtracking with choose-explore-unchoose pattern
- At each position, try all remaining unused elements
- Recursively build permutation by fixing one element at a time
- Backtrack by removing the element when done exploring

**Time Complexity**: O(n! × n) | **Space Complexity**: O(n) for recursion

In [ ]:
def permute(nums):
    """
    Generate all permutations of given numbers.
    
    Args:
        nums: List of numbers
    
    Returns:
        List of all permutations
    """
    result = []
    
    def backtrack(current_permutation):
        # Base case: we have a complete permutation
        if len(current_permutation) == len(nums):
            result.append(current_permutation[:])
            return
        
        # Try each number that hasn't been used yet
        for num in nums:
            if num not in current_permutation:
                # Choose
                current_permutation.append(num)
                
                # Explore
                backtrack(current_permutation)
                
                # Unchoose (backtrack)
                current_permutation.pop()
    
    backtrack([])
    return result

def permute_optimized(nums):
    """
    More efficient version using index swapping.
    """
    result = []
    
    def backtrack(start_index):
        # Base case: we've fixed all positions
        if start_index == len(nums):
            result.append(nums[:])
            return
        
        # Try each element in remaining positions
        for i in range(start_index, len(nums)):
            # Choose: swap current element to fixed position
            nums[start_index], nums[i] = nums[i], nums[start_index]
            
            # Explore
            backtrack(start_index + 1)
            
            # Unchoose: swap back
            nums[start_index], nums[i] = nums[i], nums[start_index]
    
    backtrack(0)
    return result

def permute_unique(nums):
    """
    Generate unique permutations when array contains duplicates.
    
    Args:
        nums: List of numbers (may contain duplicates)
    
    Returns:
        List of unique permutations
    """
    result = []
    nums.sort()  # Sort to group duplicates together
    used = [False] * len(nums)
    
    def backtrack(current_permutation):
        if len(current_permutation) == len(nums):
            result.append(current_permutation[:])
            return
        
        for i in range(len(nums)):
            # Skip if already used
            if used[i]:
                continue
            
            # Skip duplicates: if current number equals previous and previous not used
            if i > 0 and nums[i] == nums[i-1] and not used[i-1]:
                continue
            
            # Choose
            current_permutation.append(nums[i])
            used[i] = True
            
            # Explore
            backtrack(current_permutation)
            
            # Unchoose
            current_permutation.pop()
            used[i] = False
    
    backtrack([])
    return result

def next_permutation(nums):
    """
    Find the next lexicographically greater permutation.
    If no such permutation exists, rearrange to smallest permutation.
    
    Args:
        nums: List to modify in-place
    """
    # Find the largest index i such that nums[i] < nums[i + 1]
    i = len(nums) - 2
    while i >= 0 and nums[i] >= nums[i + 1]:
        i -= 1
    
    if i >= 0:  # If such index exists
        # Find the largest index j such that nums[i] < nums[j]
        j = len(nums) - 1
        while nums[j] <= nums[i]:
            j -= 1
        # Swap nums[i] and nums[j]
        nums[i], nums[j] = nums[j], nums[i]
    
    # Reverse the suffix starting at nums[i + 1]
    nums[i + 1:] = reversed(nums[i + 1:])

# Test permutation functions
print("=== Permutation Problems ===")

# Test basic permutations
test_arrays = [
    [1, 2, 3],
    [0, 1],
    [1],
    [1, 2, 3, 4]  # Warning: 4! = 24 permutations
]

for i, nums in enumerate(test_arrays[:-1]):  # Skip the large one for now
    print(f"\nTest {i+1}: {nums}")
    
    perms1 = permute(nums)
    perms2 = permute_optimized(nums.copy())
    
    print(f"Permutations (method 1): {perms1}")
    print(f"Permutations (method 2): {perms2}")
    print(f"Count: {len(perms1)}")
    print(f"Expected count: {len(nums)}! = {1 if not nums else eval('*'.join(map(str, range(1, len(nums)+1))))}")

# Test permutations with duplicates
print("\nPermutations with duplicates:")
duplicate_tests = [
    [1, 1, 2],
    [1, 2, 1],
    [2, 2, 1, 1],
]

for nums in duplicate_tests:
    unique_perms = permute_unique(nums)
    print(f"  {nums}: {len(unique_perms)} unique permutations")
    print(f"    {unique_perms}")

# Test next permutation
print("\nNext permutation:")
next_perm_tests = [
    [1, 2, 3],    # -> [1, 3, 2]
    [3, 2, 1],    # -> [1, 2, 3] (wrap around)
    [1, 1, 5],    # -> [1, 5, 1]
    [1, 3, 2],    # -> [2, 1, 3]
]

for nums in next_perm_tests:
    original = nums.copy()
    next_permutation(nums)
    print(f"  {original} -> {nums}")

## Problem 2: Generate Combinations

**Problem**: Generate all possible combinations of k numbers from 1 to n.

**Approach**: Backtracking with early termination
- Build combinations by choosing numbers in ascending order
- Use start index to avoid duplicates and ensure ordering
- Prune branches when remaining elements insufficient

**Time Complexity**: O(C(n,k) × k) | **Space Complexity**: O(k) for recursion

In [ ]:
def combine(n, k):
    """
    Generate all combinations of k numbers from 1 to n.
    
    Args:
        n: Upper limit (inclusive)
        k: Number of elements to choose
    
    Returns:
        List of all combinations
    """
    result = []
    
    def backtrack(start, current_combination):
        # Base case: we have k elements
        if len(current_combination) == k:
            result.append(current_combination[:])
            return
        
        # Optimization: early termination if not enough elements left
        needed = k - len(current_combination)
        available = n - start + 1
        if available < needed:
            return
        
        # Try each number starting from 'start'
        for i in range(start, n + 1):
            # Choose
            current_combination.append(i)
            
            # Explore (next number must be greater than current)
            backtrack(i + 1, current_combination)
            
            # Unchoose
            current_combination.pop()
    
    backtrack(1, [])
    return result

def combination_sum(candidates, target):
    """
    Find all unique combinations where candidate numbers sum to target.
    Same number can be used multiple times.
    
    Args:
        candidates: List of distinct integers
        target: Target sum
    
    Returns:
        List of all unique combinations that sum to target
    """
    result = []
    candidates.sort()  # Sort for optimization
    
    def backtrack(start, current_combination, remaining):
        if remaining == 0:
            result.append(current_combination[:])
            return
        
        for i in range(start, len(candidates)):
            candidate = candidates[i]
            
            # Pruning: if current candidate is too large, all subsequent will be too
            if candidate > remaining:
                break
            
            # Choose
            current_combination.append(candidate)
            
            # Explore (can reuse same candidate, so start from i)
            backtrack(i, current_combination, remaining - candidate)
            
            # Unchoose
            current_combination.pop()
    
    backtrack(0, [], target)
    return result

def combination_sum_ii(candidates, target):
    """
    Find combinations that sum to target, each number used at most once.
    Array may contain duplicates.
    
    Args:
        candidates: List of integers (may contain duplicates)
        target: Target sum
    
    Returns:
        List of unique combinations
    """
    result = []
    candidates.sort()
    
    def backtrack(start, current_combination, remaining):
        if remaining == 0:
            result.append(current_combination[:])
            return
        
        for i in range(start, len(candidates)):
            candidate = candidates[i]
            
            # Pruning
            if candidate > remaining:
                break
            
            # Skip duplicates at same level
            if i > start and candidates[i] == candidates[i-1]:
                continue
            
            # Choose
            current_combination.append(candidate)
            
            # Explore (move to next position since each number used once)
            backtrack(i + 1, current_combination, remaining - candidate)
            
            # Unchoose
            current_combination.pop()
    
    backtrack(0, [], target)
    return result

def letter_combinations(digits):
    """
    Letter combinations of a phone number.
    
    Args:
        digits: String containing digits 2-9
    
    Returns:
        List of all possible letter combinations
    """
    if not digits:
        return []
    
    phone_map = {
        '2': 'abc', '3': 'def', '4': 'ghi', '5': 'jkl',
        '6': 'mno', '7': 'pqrs', '8': 'tuv', '9': 'wxyz'
    }
    
    result = []
    
    def backtrack(index, current_combination):
        # Base case: processed all digits
        if index == len(digits):
            result.append(current_combination)
            return
        
        # Get letters for current digit
        letters = phone_map[digits[index]]
        
        # Try each letter
        for letter in letters:
            # Choose and explore (no need to unchoose with string concatenation)
            backtrack(index + 1, current_combination + letter)
    
    backtrack(0, "")
    return result

# Test combination functions
print("=== Combination Problems ===")

# Test basic combinations
print("Basic Combinations C(n, k):")
combo_tests = [(4, 2), (3, 3), (5, 1), (1, 1)]

for n, k in combo_tests:
    combinations = combine(n, k)
    print(f"  C({n}, {k}) = {len(combinations)} combinations")
    print(f"    {combinations}")
    
    # Calculate expected count: n! / (k! * (n-k)!)
    import math
    expected = math.comb(n, k) if hasattr(math, 'comb') else 'N/A'
    print(f"    Expected: {expected}")

# Test combination sum
print("\nCombination Sum (with repetition):")
sum_tests = [
    ([2, 3, 6, 7], 7),      # Expected: [[2,2,3], [7]]
    ([2, 3, 5], 8),         # Expected: [[2,2,2,2], [2,3,3], [3,5]]
    ([2], 1),               # Expected: [] (impossible)
]

for candidates, target in sum_tests:
    combinations = combination_sum(candidates, target)
    print(f"  Candidates {candidates}, target {target}:")
    print(f"    {combinations}")
    print(f"    Count: {len(combinations)}")

# Test combination sum II
print("\nCombination Sum II (no repetition, may have duplicates):")
sum2_tests = [
    ([10, 1, 2, 7, 6, 1, 5], 8),   # Expected: [[1,1,6], [1,2,5], [1,7], [2,6]]
    ([2, 5, 2, 1, 2], 5),          # Expected: [[1,2,2], [5]]
]

for candidates, target in sum2_tests:
    combinations = combination_sum_ii(candidates, target)
    print(f"  Candidates {candidates}, target {target}:")
    print(f"    {combinations}")
    print(f"    Count: {len(combinations)}")

# Test letter combinations
print("\nLetter Combinations of Phone Number:")
phone_tests = ['23', '2', '', '234']

for digits in phone_tests:
    if digits:  # Skip empty string for cleaner output
        combinations = letter_combinations(digits)
        print(f"  Digits '{digits}': {combinations}")
        print(f"    Count: {len(combinations)}")
        
        # Calculate expected count
        phone_map = {'2': 3, '3': 3, '4': 3, '5': 3, '6': 3, '7': 4, '8': 3, '9': 4}
        expected = 1
        for digit in digits:
            expected *= phone_map.get(digit, 1)
        print(f"    Expected: {expected}")

## Problem 3: Generate Subsets (Power Set)

**Problem**: Generate all possible subsets of a given set.

**Approach**: For each element, decide whether to include it or not
- At each step, make binary choice: include or exclude current element
- Generate 2^n total subsets
- Can also use iterative bit manipulation approach

**Time Complexity**: O(2^n × n) | **Space Complexity**: O(n) for recursion

In [ ]:
def subsets(nums):
    """
    Generate all possible subsets (power set).
    
    Args:
        nums: List of distinct integers
    
    Returns:
        List of all subsets
    """
    result = []
    
    def backtrack(index, current_subset):
        # Add current subset to result
        result.append(current_subset[:])
        
        # Try including each remaining element
        for i in range(index, len(nums)):
            # Choose
            current_subset.append(nums[i])
            
            # Explore
            backtrack(i + 1, current_subset)
            
            # Unchoose
            current_subset.pop()
    
    backtrack(0, [])
    return result

def subsets_iterative(nums):
    """
    Iterative approach: for each new element, double the existing subsets.
    """
    result = [[]]
    
    for num in nums:
        # Create new subsets by adding current number to existing subsets
        new_subsets = [subset + [num] for subset in result]
        result.extend(new_subsets)
    
    return result

def subsets_bit_manipulation(nums):
    """
    Bit manipulation approach: each subset corresponds to a binary number.
    """
    n = len(nums)
    result = []
    
    # Generate all numbers from 0 to 2^n - 1
    for mask in range(1 << n):  # 2^n possibilities
        subset = []
        for i in range(n):
            # Check if i-th bit is set
            if mask & (1 << i):
                subset.append(nums[i])
        result.append(subset)
    
    return result

def subsets_with_dup(nums):
    """
    Generate subsets when array contains duplicates.
    
    Args:
        nums: List of integers (may contain duplicates)
    
    Returns:
        List of unique subsets
    """
    result = []
    nums.sort()  # Sort to group duplicates
    
    def backtrack(index, current_subset):
        result.append(current_subset[:])
        
        for i in range(index, len(nums)):
            # Skip duplicates at same level
            if i > index and nums[i] == nums[i-1]:
                continue
            
            # Choose
            current_subset.append(nums[i])
            
            # Explore
            backtrack(i + 1, current_subset)
            
            # Unchoose
            current_subset.pop()
    
    backtrack(0, [])
    return result

def generate_parentheses(n):
    """
    Generate all combinations of well-formed parentheses.
    
    Args:
        n: Number of pairs of parentheses
    
    Returns:
        List of all valid parentheses combinations
    """
    result = []
    
    def backtrack(current, open_count, close_count):
        # Base case: used all parentheses
        if len(current) == 2 * n:
            result.append(current)
            return
        
        # Add opening parenthesis if we haven't used all
        if open_count < n:
            backtrack(current + '(', open_count + 1, close_count)
        
        # Add closing parenthesis if it won't make string invalid
        if close_count < open_count:
            backtrack(current + ')', open_count, close_count + 1)
    
    backtrack('', 0, 0)
    return result

def restore_ip_addresses(s):
    """
    Restore IP addresses from a string of digits.
    
    Args:
        s: String containing only digits
    
    Returns:
        List of all possible valid IP addresses
    """
    result = []
    
    def is_valid_part(part):
        """Check if a part is valid IP address component."""
        if not part or len(part) > 3:
            return False
        if len(part) > 1 and part[0] == '0':  # No leading zeros
            return False
        return 0 <= int(part) <= 255
    
    def backtrack(start, current_parts):
        # Base case: we have 4 parts and used all characters
        if len(current_parts) == 4:
            if start == len(s):
                result.append('.'.join(current_parts))
            return
        
        # Try different lengths for next part (1-3 characters)
        for end in range(start + 1, min(start + 4, len(s) + 1)):
            part = s[start:end]
            if is_valid_part(part):
                # Choose
                current_parts.append(part)
                
                # Explore
                backtrack(end, current_parts)
                
                # Unchoose
                current_parts.pop()
    
    backtrack(0, [])
    return result

# Test subset generation functions
print("=== Subset Generation Problems ===")

# Test basic subsets
print("Basic Subsets (Power Set):")
subset_tests = [[1, 2, 3], [0], [1, 2], []]

for nums in subset_tests:
    if len(nums) <= 3:  # Only test small sets for readability
        print(f"\n  Input: {nums}")
        
        subsets1 = subsets(nums)
        subsets2 = subsets_iterative(nums)
        subsets3 = subsets_bit_manipulation(nums)
        
        print(f"    Backtracking: {subsets1}")
        print(f"    Iterative: {subsets2}")
        print(f"    Bit manipulation: {subsets3}")
        print(f"    Count: {len(subsets1)} (expected: 2^{len(nums)} = {2**len(nums)})")

# Test subsets with duplicates
print("\nSubsets with Duplicates:")
dup_tests = [[1, 2, 2], [4, 4, 4, 1, 4]]

for nums in dup_tests:
    if len(nums) <= 5:  # Keep output manageable
        unique_subsets = subsets_with_dup(nums)
        print(f"  Input: {nums}")
        print(f"    Unique subsets: {unique_subsets}")
        print(f"    Count: {len(unique_subsets)}")

# Test generate parentheses
print("\nGenerate Valid Parentheses:")
paren_tests = [1, 2, 3, 4]

for n in paren_tests:
    if n <= 3:  # Keep output manageable
        parentheses = generate_parentheses(n)
        print(f"  n = {n}: {parentheses}")
        print(f"    Count: {len(parentheses)}")
        
        # Verify all are valid
        all_valid = all(
            s.count('(') == s.count(')') == n and
            all(s[:i+1].count('(') >= s[:i+1].count(')') for i in range(len(s)))
            for s in parentheses
        )
        print(f"    All valid: {all_valid}")

# Test IP address restoration
print("\nRestore IP Addresses:")
ip_tests = ['25525511135', '0000', '101023']

for s in ip_tests:
    ip_addresses = restore_ip_addresses(s)
    print(f"  Input: '{s}'")
    print(f"    Valid IP addresses: {ip_addresses}")
    print(f"    Count: {len(ip_addresses)}")

## Problem 4: N-Queens Problem

**Problem**: Place N queens on an N×N chessboard so that no two queens attack each other.

**Approach**: Classic constraint satisfaction with backtracking
- Place queens row by row
- For each row, try each column position
- Check if position is safe (no conflicts with existing queens)
- Use optimized conflict checking

**Time Complexity**: O(N!) | **Space Complexity**: O(N)

In [ ]:
def solve_n_queens(n):
    """
    Solve N-Queens problem and return all solutions.
    
    Args:
        n: Size of the chessboard (n x n)
    
    Returns:
        List of all solution boards
    """
    result = []
    board = ['.' * n for _ in range(n)]
    
    def is_safe(row, col):
        """Check if placing queen at (row, col) is safe."""
        # Check column
        for i in range(row):
            if board[i][col] == 'Q':
                return False
        
        # Check upper diagonal (left)
        for i, j in zip(range(row-1, -1, -1), range(col-1, -1, -1)):
            if board[i][j] == 'Q':
                return False
        
        # Check upper diagonal (right)
        for i, j in zip(range(row-1, -1, -1), range(col+1, n)):
            if board[i][j] == 'Q':
                return False
        
        return True
    
    def backtrack(row):
        # Base case: placed all queens
        if row == n:
            result.append([''.join(row) for row in board])
            return
        
        # Try each column in current row
        for col in range(n):
            if is_safe(row, col):
                # Choose
                board[row] = board[row][:col] + 'Q' + board[row][col+1:]
                
                # Explore
                backtrack(row + 1)
                
                # Unchoose
                board[row] = board[row][:col] + '.' + board[row][col+1:]
    
    backtrack(0)
    return result

def solve_n_queens_optimized(n):
    """
    Optimized version using sets for O(1) conflict checking.
    """
    result = []
    board = ['.' * n for _ in range(n)]
    
    # Track occupied columns and diagonals
    cols = set()
    diag1 = set()  # row - col
    diag2 = set()  # row + col
    
    def backtrack(row):
        if row == n:
            result.append([''.join(row) for row in board])
            return
        
        for col in range(n):
            if col in cols or (row - col) in diag1 or (row + col) in diag2:
                continue
            
            # Choose
            board[row] = board[row][:col] + 'Q' + board[row][col+1:]
            cols.add(col)
            diag1.add(row - col)
            diag2.add(row + col)
            
            # Explore
            backtrack(row + 1)
            
            # Unchoose
            board[row] = board[row][:col] + '.' + board[row][col+1:]
            cols.remove(col)
            diag1.remove(row - col)
            diag2.remove(row + col)
    
    backtrack(0)
    return result

def total_n_queens(n):
    """
    Count total number of N-Queens solutions without storing them.
    """
    count = [0]  # Use list for mutable counter
    
    cols = set()
    diag1 = set()
    diag2 = set()
    
    def backtrack(row):
        if row == n:
            count[0] += 1
            return
        
        for col in range(n):
            if col in cols or (row - col) in diag1 or (row + col) in diag2:
                continue
            
            # Choose
            cols.add(col)
            diag1.add(row - col)
            diag2.add(row + col)
            
            # Explore
            backtrack(row + 1)
            
            # Unchoose
            cols.remove(col)
            diag1.remove(row - col)
            diag2.remove(row + col)
    
    backtrack(0)
    return count[0]

def print_board(board):
    """Helper function to print a chess board nicely."""
    for row in board:
        print('  ' + ' '.join(row))
    print()

def is_valid_sudoku_helper(board):
    """
    Helper function to check if current Sudoku state is valid.
    """
    # Check rows
    for row in board:
        seen = set()
        for cell in row:
            if cell != '.' and cell in seen:
                return False
            if cell != '.':
                seen.add(cell)
    
    # Check columns
    for col in range(9):
        seen = set()
        for row in range(9):
            cell = board[row][col]
            if cell != '.' and cell in seen:
                return False
            if cell != '.':
                seen.add(cell)
    
    # Check 3x3 boxes
    for box_row in range(0, 9, 3):
        for box_col in range(0, 9, 3):
            seen = set()
            for row in range(box_row, box_row + 3):
                for col in range(box_col, box_col + 3):
                    cell = board[row][col]
                    if cell != '.' and cell in seen:
                        return False
                    if cell != '.':
                        seen.add(cell)
    
    return True

def solve_sudoku(board):
    """
    Solve Sudoku puzzle using backtracking.
    Modifies board in-place.
    
    Args:
        board: 9x9 grid with '.' for empty cells
    
    Returns:
        True if solved, False if no solution
    """
    def is_valid(row, col, num):
        """Check if placing num at (row, col) is valid."""
        # Check row
        for c in range(9):
            if board[row][c] == num:
                return False
        
        # Check column
        for r in range(9):
            if board[r][col] == num:
                return False
        
        # Check 3x3 box
        box_row, box_col = 3 * (row // 3), 3 * (col // 3)
        for r in range(box_row, box_row + 3):
            for c in range(box_col, box_col + 3):
                if board[r][c] == num:
                    return False
        
        return True
    
    def solve():
        for row in range(9):
            for col in range(9):
                if board[row][col] == '.':
                    # Try digits 1-9
                    for num in '123456789':
                        if is_valid(row, col, num):
                            # Choose
                            board[row][col] = num
                            
                            # Explore
                            if solve():
                                return True
                            
                            # Unchoose
                            board[row][col] = '.'
                    
                    return False  # No valid digit found
        return True  # All cells filled
    
    return solve()

# Test N-Queens and Sudoku
print("=== Constraint Satisfaction Problems ===")

# Test N-Queens
print("N-Queens Problem:")
for n in range(1, 6):  # Test small sizes
    solutions = solve_n_queens_optimized(n)
    count = total_n_queens(n)
    print(f"\n  {n}-Queens: {len(solutions)} solutions (count verification: {count})")
    
    if n <= 4 and solutions:  # Show first solution for small boards
        print(f"    First solution:")
        print_board(solutions[0])

# Test Sudoku solver with a simple puzzle
print("\nSudoku Solver:")
simple_sudoku = [
    ['5', '3', '.', '.', '7', '.', '.', '.', '.'],
    ['6', '.', '.', '1', '9', '5', '.', '.', '.'],
    ['.', '9', '8', '.', '.', '.', '.', '6', '.'],
    ['8', '.', '.', '.', '6', '.', '.', '.', '3'],
    ['4', '.', '.', '8', '.', '3', '.', '.', '1'],
    ['7', '.', '.', '.', '2', '.', '.', '.', '6'],
    ['.', '6', '.', '.', '.', '.', '2', '8', '.'],
    ['.', '.', '.', '4', '1', '9', '.', '.', '5'],
    ['.', '.', '.', '.', '8', '.', '.', '7', '9']
]

print("  Original puzzle:")
for row in simple_sudoku:
    print('    ' + ' '.join(row))

if solve_sudoku(simple_sudoku):
    print("\n  Solved puzzle:")
    for row in simple_sudoku:
        print('    ' + ' '.join(row))
    
    # Verify solution is valid
    is_valid = is_valid_sudoku_helper(simple_sudoku)
    print(f"\n  Solution is valid: {is_valid}")
else:
    print("\n  No solution found!")

## Problem 5: Word Search and Path Finding

**Problem**: Find if a word exists in a 2D grid by connecting adjacent letters.

**Approach**: DFS with backtracking on 2D grid
- Start from each cell that matches first letter
- Use DFS to explore all 4 directions
- Mark visited cells temporarily to avoid reuse
- Backtrack by unmarking when returning from recursion

**Time Complexity**: O(m×n×4^L) | **Space Complexity**: O(L) where L is word length

In [ ]:
def word_search(board, word):
    """
    Find if word exists in 2D character grid.
    
    Args:
        board: 2D grid of characters
        word: Word to search for
    
    Returns:
        True if word exists, False otherwise
    """
    if not board or not board[0] or not word:
        return False
    
    rows, cols = len(board), len(board[0])
    
    def dfs(row, col, word_index):
        # Base case: found complete word
        if word_index == len(word):
            return True
        
        # Check boundaries and character match
        if (row < 0 or row >= rows or col < 0 or col >= cols or
            board[row][col] != word[word_index] or board[row][col] == '#'):
            return False
        
        # Choose: mark cell as visited
        temp = board[row][col]
        board[row][col] = '#'  # Mark as visited
        
        # Explore all 4 directions
        found = (dfs(row + 1, col, word_index + 1) or
                dfs(row - 1, col, word_index + 1) or
                dfs(row, col + 1, word_index + 1) or
                dfs(row, col - 1, word_index + 1))
        
        # Unchoose: restore original character
        board[row][col] = temp
        
        return found
    
    # Try starting from each cell
    for row in range(rows):
        for col in range(cols):
            if dfs(row, col, 0):
                return True
    
    return False

def word_search_ii(board, words):
    """
    Find all words that can be formed in the grid.
    More efficient than calling word_search for each word.
    
    Args:
        board: 2D grid of characters
        words: List of words to search for
    
    Returns:
        List of words found in the grid
    """
    class TrieNode:
        def __init__(self):
            self.children = {}
            self.word = None
    
    # Build trie from words
    root = TrieNode()
    for word in words:
        node = root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.word = word
    
    result = set()
    rows, cols = len(board), len(board[0])
    
    def dfs(row, col, node):
        if row < 0 or row >= rows or col < 0 or col >= cols:
            return
        
        char = board[row][col]
        if char == '#' or char not in node.children:
            return
        
        # Move to next node in trie
        node = node.children[char]
        
        # Check if we found a word
        if node.word:
            result.add(node.word)
        
        # Choose: mark as visited
        board[row][col] = '#'
        
        # Explore all 4 directions
        for dr, dc in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            dfs(row + dr, col + dc, node)
        
        # Unchoose: restore character
        board[row][col] = char
    
    # Start DFS from each cell
    for row in range(rows):
        for col in range(cols):
            dfs(row, col, root)
    
    return list(result)

def solve_maze(maze, start, end):
    """
    Find path through maze using backtracking.
    
    Args:
        maze: 2D grid where 0=path, 1=wall
        start: Starting position (row, col)
        end: Ending position (row, col)
    
    Returns:
        Path from start to end, or None if no path exists
    """
    if not maze or maze[start[0]][start[1]] == 1 or maze[end[0]][end[1]] == 1:
        return None
    
    rows, cols = len(maze), len(maze[0])
    path = []
    
    def dfs(row, col):
        # Check boundaries and walls
        if (row < 0 or row >= rows or col < 0 or col >= cols or
            maze[row][col] == 1):  # 1 = wall or visited
            return False
        
        # Choose: add to path and mark as visited
        path.append((row, col))
        maze[row][col] = 1  # Mark as visited
        
        # Check if reached destination
        if (row, col) == end:
            return True
        
        # Explore all 4 directions
        if (dfs(row + 1, col) or dfs(row - 1, col) or
            dfs(row, col + 1) or dfs(row, col - 1)):
            return True
        
        # Unchoose: backtrack
        path.pop()
        maze[row][col] = 0  # Unmark
        return False
    
    if dfs(start[0], start[1]):
        return path
    return None

def find_words_with_pattern(board, pattern):
    """
    Find all words that match a pattern (with wildcards).
    '.' can match any character.
    
    Args:
        board: 2D character grid
        pattern: Pattern with possible '.' wildcards
    
    Returns:
        List of all matching words found
    """
    if not board or not board[0] or not pattern:
        return []
    
    rows, cols = len(board), len(board[0])
    found_words = set()
    
    def dfs(row, col, word_index, current_word):
        # Base case: found complete word
        if word_index == len(pattern):
            found_words.add(current_word)
            return
        
        # Check boundaries
        if row < 0 or row >= rows or col < 0 or col >= cols or board[row][col] == '#':
            return
        
        char = board[row][col]
        
        # Check if character matches pattern
        if pattern[word_index] != '.' and pattern[word_index] != char:
            return
        
        # Choose
        board[row][col] = '#'  # Mark as visited
        
        # Explore all 4 directions
        for dr, dc in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            dfs(row + dr, col + dc, word_index + 1, current_word + char)
        
        # Unchoose
        board[row][col] = char
    
    # Try starting from each cell
    for row in range(rows):
        for col in range(cols):
            dfs(row, col, 0, '')
    
    return list(found_words)

# Test word search and path finding
print("=== Word Search and Path Finding ===")

# Test basic word search
print("Word Search:")
test_board = [
    ['A', 'B', 'C', 'E'],
    ['S', 'F', 'C', 'S'],
    ['A', 'D', 'E', 'E']
]

test_words = ['ABCCED', 'SEE', 'ABCB', 'ASA', 'ASADFCCS']

print("  Board:")
for row in test_board:
    print(f"    {row}")

for word in test_words:
    # Make a copy since word_search modifies the board
    board_copy = [row[:] for row in test_board]
    found = word_search(board_copy, word)
    print(f"    Word '{word}': {'Found' if found else 'Not found'}")

# Test word search II (multiple words)
print("\nWord Search II (Multiple Words):")
words_to_find = ['ABCCED', 'SEE', 'ABCB']
board_copy = [row[:] for row in test_board]
found_words = word_search_ii(board_copy, words_to_find)
print(f"  Searching for: {words_to_find}")
print(f"  Found: {found_words}")

# Test maze solving
print("\nMaze Solving:")
maze = [
    [0, 1, 0, 0, 0],
    [0, 1, 0, 1, 0],
    [0, 0, 0, 1, 0],
    [1, 1, 0, 0, 0],
    [0, 0, 0, 1, 0]
]

print("  Maze (0=path, 1=wall):")
for row in maze:
    print(f"    {row}")

start, end = (0, 0), (4, 4)
maze_copy = [row[:] for row in maze]
path = solve_maze(maze_copy, start, end)
print(f"  Start: {start}, End: {end}")
if path:
    print(f"  Path found: {path}")
    print(f"  Path length: {len(path)}")
else:
    print("  No path found")

# Test pattern matching
print("\nPattern Matching with Wildcards:")
pattern_board = [
    ['C', 'A', 'T'],
    ['O', 'G', 'S'],
    ['D', 'O', 'G']
]

print("  Board:")
for row in pattern_board:
    print(f"    {row}")

patterns = ['CAT', 'DOG', 'C.T', 'D.G', '...']
for pattern in patterns:
    board_copy = [row[:] for row in pattern_board]
    matches = find_words_with_pattern(board_copy, pattern)
    print(f"  Pattern '{pattern}': {matches}")

## Problem 6: Palindrome Partitioning

**Problem**: Partition a string into substrings where each substring is a palindrome.

**Approach**: Backtracking with palindrome checking
- At each position, try all possible partitions
- Check if current substring is a palindrome
- Use dynamic programming for efficient palindrome checking
- Recursively partition the remaining string

**Time Complexity**: O(N×2^N) | **Space Complexity**: O(N)

In [ ]:
def partition_palindrome(s):
    """
    Partition string into palindromic substrings.
    
    Args:
        s: Input string
    
    Returns:
        List of all possible palindrome partitions
    """
    result = []
    n = len(s)
    
    # Precompute palindrome table for efficiency
    is_palindrome = [[False] * n for _ in range(n)]
    
    # Every single character is a palindrome
    for i in range(n):
        is_palindrome[i][i] = True
    
    # Check for palindromes of length 2
    for i in range(n - 1):
        if s[i] == s[i + 1]:
            is_palindrome[i][i + 1] = True
    
    # Check for palindromes of length 3 and more
    for length in range(3, n + 1):
        for i in range(n - length + 1):
            j = i + length - 1
            if s[i] == s[j] and is_palindrome[i + 1][j - 1]:
                is_palindrome[i][j] = True
    
    def backtrack(start, current_partition):
        # Base case: reached end of string
        if start == len(s):
            result.append(current_partition[:])
            return
        
        # Try all possible end positions
        for end in range(start, len(s)):
            if is_palindrome[start][end]:
                # Choose
                current_partition.append(s[start:end + 1])
                
                # Explore
                backtrack(end + 1, current_partition)
                
                # Unchoose
                current_partition.pop()
    
    backtrack(0, [])
    return result

def min_palindrome_partitions(s):
    """
    Find minimum number of cuts to partition string into palindromes.
    
    Args:
        s: Input string
    
    Returns:
        Minimum number of cuts needed
    """
    n = len(s)
    
    # Precompute palindrome table
    is_palindrome = [[False] * n for _ in range(n)]
    
    for i in range(n):
        is_palindrome[i][i] = True
    
    for i in range(n - 1):
        if s[i] == s[i + 1]:
            is_palindrome[i][i + 1] = True
    
    for length in range(3, n + 1):
        for i in range(n - length + 1):
            j = i + length - 1
            if s[i] == s[j] and is_palindrome[i + 1][j - 1]:
                is_palindrome[i][j] = True
    
    # DP to find minimum cuts
    cuts = [float('inf')] * n
    
    for i in range(n):
        if is_palindrome[0][i]:
            cuts[i] = 0  # No cut needed
        else:
            for j in range(i):
                if is_palindrome[j + 1][i]:
                    cuts[i] = min(cuts[i], cuts[j] + 1)
    
    return cuts[n - 1]

def add_operators(num, target):
    """
    Add operators (+, -, *) between digits to reach target value.
    
    Args:
        num: String of digits
        target: Target value
    
    Returns:
        List of all valid expressions
    """
    result = []
    
    def backtrack(index, expression, value, prev_num):
        # Base case: processed all digits
        if index == len(num):
            if value == target:
                result.append(expression)
            return
        
        # Try all possible number lengths from current position
        for i in range(index, len(num)):
            curr_str = num[index:i + 1]
            curr_num = int(curr_str)
            
            # Skip numbers with leading zeros (except single '0')
            if len(curr_str) > 1 and curr_str[0] == '0':
                break
            
            if index == 0:
                # First number, no operator needed
                backtrack(i + 1, curr_str, curr_num, curr_num)
            else:
                # Try addition
                backtrack(i + 1, expression + '+' + curr_str, 
                         value + curr_num, curr_num)
                
                # Try subtraction
                backtrack(i + 1, expression + '-' + curr_str, 
                         value - curr_num, -curr_num)
                
                # Try multiplication (need to undo previous operation)
                backtrack(i + 1, expression + '*' + curr_str, 
                         value - prev_num + prev_num * curr_num, 
                         prev_num * curr_num)
    
    backtrack(0, '', 0, 0)
    return result

def remove_invalid_parentheses(s):
    """
    Remove minimum number of parentheses to make string valid.
    
    Args:
        s: String with parentheses
    
    Returns:
        List of all valid strings with minimum removals
    """
    def is_valid(string):
        count = 0
        for char in string:
            if char == '(':
                count += 1
            elif char == ')':
                count -= 1
                if count < 0:
                    return False
        return count == 0
    
    def get_min_removals(s):
        left_rem = right_rem = 0
        for char in s:
            if char == '(':
                left_rem += 1
            elif char == ')':
                if left_rem > 0:
                    left_rem -= 1
                else:
                    right_rem += 1
        return left_rem, right_rem
    
    result = set()
    left_rem, right_rem = get_min_removals(s)
    
    def backtrack(index, current, left_count, right_count, left_rem, right_rem):
        if index == len(s):
            if left_rem == 0 and right_rem == 0 and is_valid(current):
                result.add(current)
            return
        
        char = s[index]
        
        # Option 1: Remove current character
        if char == '(' and left_rem > 0:
            backtrack(index + 1, current, left_count, right_count, left_rem - 1, right_rem)
        elif char == ')' and right_rem > 0:
            backtrack(index + 1, current, left_count, right_count, left_rem, right_rem - 1)
        
        # Option 2: Keep current character
        current += char
        if char == '(':
            backtrack(index + 1, current, left_count + 1, right_count, left_rem, right_rem)
        elif char == ')':
            if left_count > right_count:  # Valid to add ')'
                backtrack(index + 1, current, left_count, right_count + 1, left_rem, right_rem)
        else:  # Regular character
            backtrack(index + 1, current, left_count, right_count, left_rem, right_rem)
    
    backtrack(0, '', 0, 0, left_rem, right_rem)
    return list(result) if result else ['']

# Test advanced backtracking problems
print("=== Advanced Backtracking Problems ===")

# Test palindrome partitioning
print("Palindrome Partitioning:")
partition_tests = ['aab', 'raceacar', 'abcde', 'aba']

for s in partition_tests:
    if len(s) <= 6:  # Keep output manageable
        partitions = partition_palindrome(s)
        min_cuts = min_palindrome_partitions(s)
        print(f"  String '{s}':")
        print(f"    All partitions: {partitions}")
        print(f"    Count: {len(partitions)}")
        print(f"    Min cuts needed: {min_cuts}")

# Test expression evaluation
print("\nAdd Operators to Reach Target:")
operator_tests = [
    ('123', 6),      # Expected: ['1+2+3', '1*2*3']
    ('232', 8),      # Expected: ['2*3+2', '2+3*2']
    ('105', 5),      # Expected: ['1*0+5', '10-5']
    ('00', 0),       # Expected: ['0+0', '0-0', '0*0']
]

for num, target in operator_tests:
    if len(num) <= 4:  # Keep computation reasonable
        expressions = add_operators(num, target)
        print(f"  '{num}' -> {target}: {expressions}")
        print(f"    Count: {len(expressions)}")

# Test remove invalid parentheses
print("\nRemove Invalid Parentheses:")
paren_tests = ['()())', '(((', '((a))', ')(']

for s in paren_tests:
    valid_strings = remove_invalid_parentheses(s)
    print(f"  '{s}': {valid_strings}")
    print(f"    Count: {len(valid_strings)}")
    
    # Verify all results are valid
    def verify_valid(string):
        count = 0
        for char in string:
            if char == '(':
                count += 1
            elif char == ')':
                count -= 1
                if count < 0:
                    return False
        return count == 0
    
    all_valid = all(verify_valid(string) for string in valid_strings)
    print(f"    All results valid: {all_valid}")

## Summary and Key Takeaways

### The Backtracking Algorithm Pattern:
```python
def backtrack(state):
    if is_complete(state):
        process_solution(state)
        return
    
    for choice in get_choices(state):
        if is_valid(choice, state):
            make_choice(choice, state)     # Choose
            backtrack(new_state)           # Explore
            undo_choice(choice, state)     # Unchoose
```

### When to Use Backtracking:
1. **Generate all possibilities**: Permutations, combinations, subsets
2. **Constraint satisfaction**: N-Queens, Sudoku, graph coloring
3. **Path finding with constraints**: Word search, maze solving
4. **Optimization with exhaustive search**: When DP/greedy won't work
5. **Parsing and validation**: Expression evaluation, parentheses matching

### Problem Categories and Patterns:

#### **1. Combinatorial Problems:**
- **Permutations**: All arrangements of elements
- **Combinations**: Choose k from n elements
- **Subsets**: All possible subsets (power set)
- **Partitioning**: Split into valid groups

#### **2. Constraint Satisfaction:**
- **N-Queens**: Place queens without conflicts
- **Sudoku**: Fill grid following rules
- **Graph Coloring**: Color vertices without adjacent same colors

#### **3. Path Finding:**
- **Word Search**: Find word in 2D grid
- **Maze Solving**: Find path from start to end
- **Pattern Matching**: Match with wildcards

#### **4. String Problems:**
- **Palindrome Partitioning**: Split into palindromic substrings
- **Expression Evaluation**: Add operators to reach target
- **Parentheses Problems**: Generate/validate/remove

### Optimization Techniques:

#### **1. Pruning Strategies:**
- **Early termination**: Stop when solution impossible
- **Constraint checking**: Validate before exploring
- **Bound checking**: Use upper/lower bounds
- **Duplicate avoidance**: Skip equivalent states

#### **2. State Representation:**
- **Efficient data structures**: Sets for O(1) lookups
- **Bit manipulation**: Compact state representation
- **Precomputation**: Build lookup tables (palindromes, etc.)

#### **3. Search Order:**
- **Most constrained first**: Choose variables with fewer options
- **Least remaining values**: Explore most promising branches first
- **Fail fast**: Check constraints early

### Time/Space Complexity Patterns:
| Problem Type | Time Complexity | Space Complexity | Notes |
|--------------|----------------|------------------|-------|
| Permutations | O(n! × n) | O(n) | n! permutations, n to copy each |
| Combinations | O(C(n,k) × k) | O(k) | C(n,k) combinations |
| Subsets | O(2ⁿ × n) | O(n) | 2ⁿ subsets |
| N-Queens | O(n!) | O(n) | Exponential with pruning |
| Word Search | O(m×n×4^L) | O(L) | Grid size × path length |

### Common Pitfalls and Solutions:

#### **1. Forgetting to Backtrack:**
```python
# Wrong - no backtracking
def wrong_approach(state):
    state.append(choice)
    backtrack(state)
    # Missing: state.pop()

# Correct - proper backtracking
def correct_approach(state):
    state.append(choice)
    backtrack(state)
    state.pop()  # Undo choice
```

#### **2. Inefficient Duplicate Checking:**
```python
# Slow - O(n) lookup
if choice not in current_list:

# Fast - O(1) lookup
if choice not in current_set:
```

#### **3. Missing Early Termination:**
```python
# Add pruning conditions
if remaining_choices < needed_choices:
    return  # Impossible to complete
```

### Interview Tips:
1. **Identify the pattern**: Is this combinatorial, constraint satisfaction, or path finding?
2. **Start with brute force**: Get working solution first, optimize later
3. **Think about pruning**: What conditions allow early termination?
4. **Handle duplicates**: Sort input and skip duplicate choices when needed
5. **Verify backtracking**: Ensure all choices are properly undone

### Key Concepts Mastered:
- ✅ Choose-Explore-Unchoose pattern
- ✅ Permutations, combinations, and subsets generation
- ✅ Constraint satisfaction problems (N-Queens, Sudoku)
- ✅ 2D grid search and path finding
- ✅ String partitioning and expression problems
- ✅ Optimization techniques and pruning strategies
- ✅ Duplicate handling and state management

---

**Next Steps**: Practice identifying backtracking problems and implementing efficient pruning strategies. Master the art of state management and constraint checking!